In [2]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("dark")
plt.rcParams['figure.figsize'] = 16, 12
import pandas as pd
from tqdm import tqdm_notebook
import io
from PIL import Image
from glob import glob
from collections import defaultdict
import os
import pickle
from io import BytesIO
import random

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.datasets import ImageFolder

from kaggle_camera_model_id_lib.utils import PechkaBot, ImageList, NpzFolder, NCrops, TifFolderExFiles
from kaggle_camera_model_id_lib.models import VggHead, StyleVggHead, IEEEfcn, ResNetFC
from kaggle_camera_model_id_lib.utils import jpg_compress, equalize_v_hist, hsv_convert
from kaggle_camera_model_id_lib.utils import scale_crop_pad, gamma_correction

import imgaug as ia
from imgaug import augmenters as iaa

In [3]:
val_path = '/home/mephistopheies/storage2/data/camera-model-id/val/'
test_path = '/home/mephistopheies/storage2/data/camera-model-id/raw/test/'
model_path = '/home/mephistopheies/storage2/data/camera-model-id/models/resnet34fc/256_mml_pretrained_random_aug_kaggle_10/best_model.tar'
out_dir = '/home/mephistopheies/storage2/data/camera-model-id/submit/'
model_type = 'resnet34fc_pretrained'
n_classes = 10
crop_size = 256
step = 128
num_workers = 1

do_random_aug_kaggle = True
p_random_aug_kaggle = 1.0
do_hard_aug = False
p_hard_aug = 0.5

to_tensor = transforms.ToTensor()
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

scale_05 = lambda img: scale_crop_pad(img, 0.5)
scale_08 = lambda img: scale_crop_pad(img, 0.8)
scale_15 = lambda img: scale_crop_pad(img, 1.5)
scale_20 = lambda img: scale_crop_pad(img, 2.0)
gamma_08 = lambda img: gamma_correction(img, 0.8)
gamma_12 = lambda img: gamma_correction(img, 1.2)
jpg_70 = lambda img: jpg_compress(img, (70, 71))
jpg_90 = lambda img: jpg_compress(img, (90, 91))
augs = [scale_05, scale_08, scale_15, scale_20, gamma_08, gamma_12, jpg_70, jpg_90]

blur = iaa.GaussianBlur(sigma=(0, 2))
sharpen = iaa.Sharpen(alpha=(0, 1), lightness=(0.5, 2))
emboss = iaa.Emboss(alpha=(0, 1), strength=(0, 2))
contrast_normalization = iaa.ContrastNormalization(alpha=(0.7, 1.3))
hard_aug = iaa.OneOf([blur, sharpen, emboss, contrast_normalization])
sometimes = iaa.Sometimes(p_hard_aug, hard_aug)


def random_aug_kaggle(img, p=0.5):
    if np.random.rand() < p:
        return random.choice(augs)(img)
    return img

def aug_train(img):
    if min(img.size) > crop_center_size:
        return random_flip(random_crop(center_crop(img)))
    return random_flip(random_crop(img))

def aug_optional(img):
    if do_hard_aug:
        img = Image.fromarray(sometimes.augment_image(np.array(img)))

    if do_random_aug_kaggle:
        img = random_aug_kaggle(img, p_random_aug_kaggle)
    return img

In [4]:
model_factory = {
    'Vgg19Head_E_2b_bn': lambda n_classes: VggHead(num_classes=n_classes, vgg_key='E_2b', load_vgg_bn=True, batch_norm=True),
    'Vgg19Head_E_3b_bn': lambda n_classes: VggHead(num_classes=n_classes, vgg_key='E_3b', load_vgg_bn=True, batch_norm=True),
    'Vgg19Head_E_bn': lambda n_classes: VggHead(num_classes=n_classes, load_vgg_bn=True, vgg_key='E', batch_norm=True),
    'Vgg11Head_A_bn': lambda n_classes: VggHead(num_classes=n_classes, load_vgg_bn=True, vgg_key='A', batch_norm=True),
    'Vgg11Head_A': lambda n_classes: VggHead(num_classes=n_classes, load_vgg_bn=True, vgg_key='A', batch_norm=False),
    'StyleVggHead_bn': lambda n_classes: StyleVggHead(num_classes=n_classes, load_vgg_bn=True),
    'IEEEfcn': lambda n_classes: IEEEfcn(n_classes),
    'resnet18fc_pretrained': lambda n_classes: ResNetFC(
        models.resnet.BasicBlock, [2, 2, 2, 2], num_classes=n_classes, load_resnet='resnet18'),
    'resnet18fc': lambda n_classes: ResNetFC(
        models.resnet.BasicBlock, [2, 2, 2, 2], num_classes=n_classes, load_resnet=None),
    'resnet18X_pretrained': lambda n_classes: ResNetX(
        models.resnet.BasicBlock, [2, 2, 2, 2], num_classes=n_classes, load_resnet='resnet18'),
    'InceptionResNetV2fc_5_10_4': lambda n_classes: InceptionResNetV2fc(
        num_classes=n_classes, nun_block35=5, num_block17=10, num_block8=4),
    'InceptionResNetV2fcSmall_5_10': lambda n_classes: InceptionResNetV2fcSmall(
        num_classes=n_classes, nun_block35=5, num_block17=10),
    'resnet34fc_pretrained': lambda n_classes: ResNetFC(
        models.resnet.BasicBlock, [3, 4, 6, 3], num_classes=n_classes, load_resnet='resnet34'),
    'resnet50fc_pretrained': lambda n_classes: ResNetFC(
        models.resnet.Bottleneck, [3, 4, 6, 3], num_classes=n_classes, load_resnet='resnet50')
}

model = model_factory[model_type](n_classes)
checkpoint = torch.load(model_path)
model.load_state_dict(checkpoint['model'])
loss_train = checkpoint['loss_train']
acc_train = checkpoint['acc_train']
loss_val = checkpoint['loss_val']
acc_val = checkpoint['acc_val']
class_to_idx = checkpoint['class_to_idx']
idx2class = dict([(v, k) for (k, v) in class_to_idx.items()])
print('Last state:\n  TLoss: %0.6f\n  TAcc:  %0.4f\n  VLoss: %0.6f\n  VAcc:  %0.4f' % 
    (loss_train[-1], acc_train[-1], loss_val[-1], acc_val[-1]))
del(checkpoint)
model = model.cuda()
model = model.eval()

Last state:
  TLoss: 0.049621
  TAcc:  0.9733
  VLoss: 0.246174
  VAcc:  0.8966


In [5]:
batch_size = 15

ds_val = NpzFolder(
    val_path,
    transform=transforms.Compose([
        transforms.Lambda(lambda img: NCrops(img, crop_size=crop_size, step=step)),
        transforms.Lambda(lambda crops: torch.stack([normalize(to_tensor(aug_optional(Image.fromarray(crop)))) 
                                                     for crop in crops]))
    ]),
    target_transform=transforms.Compose([
            transforms.Lambda(lambda y: [y]*int(np.floor(1 + (512 - crop_size)/step))**2),
            transforms.Lambda(lambda ylist: torch.LongTensor(ylist))
    ]))
val_loader = torch.utils.data.DataLoader(    
    ds_val,
    batch_size=batch_size, 
    shuffle=False,
    num_workers=1, 
    pin_memory=True)


acc_val_batch = 0
for ix_batch, (X, Y) in tqdm_notebook(enumerate(val_loader), total=int(len(ds_val.imgs)/batch_size)):
    bs, ncrops, c, h, w = X.shape
    X = X.view(-1, c, h, w)
    Y = Y.view(ncrops*bs)

    X_var = Variable(X.cuda(), volatile=True)
    Y_var = Variable(Y.cuda(), volatile=True)

    log_p = model(X_var)

    acc_val_batch += ((log_p.max(1)[1] == Y_var).float().sum()/Y_var.shape[0]).data[0]

acc_val_batch /= ix_batch + 1
print(acc_val_batch)


0.8945185029506684


In [33]:
batch_size = 15

ds_val = NpzFolder(
    val_path,
    transform=transforms.Compose([
        transforms.Lambda(lambda img: NCrops(img, crop_size=crop_size, step=step)),
        transforms.Lambda(lambda crops: torch.stack([normalize(to_tensor(aug_optional(Image.fromarray(crop)))) 
                                                     for crop in crops]))
    ]))
val_loader = torch.utils.data.DataLoader(    
    ds_val,
    batch_size=batch_size, 
    shuffle=False,
    num_workers=1, 
    pin_memory=True)


# p_val = []
acc_val_batch = 0
for ix_batch, (X, Y) in tqdm_notebook(enumerate(val_loader), total=int(len(ds_val.imgs)/batch_size)):
    bs, ncrops, c, h, w = X.shape
    X = X.view(-1, c, h, w)
    X_var = Variable(X.cuda(), volatile=True)
    Y_var = Variable(Y.cuda(), volatile=True)
    log_p = model(X_var)
    log_p = log_p.view(bs, ncrops, -1)
    
    p_amax = log_p.max(2)[1]
    votes = pd.DataFrame(p_amax.cpu().data.numpy()).apply(lambda r: r.value_counts().index[0], axis=1).values
    acc_val_batch += (Y_var.cpu().data.numpy() == votes).sum()/votes.shape[0]
    

#     p = F.softmax(log_p, dim=2)
#     p = p.prod(dim=1).pow(1/p.shape[1])
#     acc_val_batch += ((p.max(1)[1] == Y_var).float().sum()/Y_var.shape[0]).data[0]
#     p_val.append(p.cpu().data.numpy())

# p_val = np.vstack(p_val)
acc_val_batch /= ix_batch + 1
print(acc_val_batch)


0.949333333333


In [44]:
ds_test = TifFolderExFiles(
    test_path,
    transform=transforms.Compose([
        transforms.Lambda(lambda img: NCrops(np.array(img), crop_size=crop_size, step=step)),
        transforms.Lambda(lambda crops: torch.stack([normalize(to_tensor(crop)) for crop in crops]))
    ]))

test_loader = torch.utils.data.DataLoader(    
    ds_test,
    batch_size=batch_size, 
    shuffle=False,
    num_workers=num_workers, 
    pin_memory=True)

res = []
p_test = {}
for X, Y, files in tqdm_notebook(test_loader, total=int(len(ds_test.imgs)/batch_size)):
    files = list(map(lambda s: os.path.basename(s), files))
    bs, ncrops, c, h, w = X.shape
    X = X.view(-1, c, h, w)
    X_var = Variable(X.cuda(), volatile=True)
    log_p = model(X_var)
    log_p = log_p.view(bs, ncrops, -1)
    
    p_amax = log_p.max(2)[1]
    votes = pd.DataFrame(p_amax.cpu().data.numpy()).apply(lambda r: r.value_counts().index[0], axis=1).values
    res.extend(list(zip(files, [idx2class[ix] for ix in votes])))
    
#     p = F.softmax(log_p, dim=2) #.mean(dim=1)
#     p = p.prod(dim=1).pow(1/p.shape[1])
#     ix_argmax = p.max(1)[1].cpu().data.numpy()
#     res.extend(list(zip(files, [idx2class[ix] for ix in ix_argmax])))

#     for ix in range(len(files)):
#         p_test[files[ix]] = [(idx2class[i], x) for (i, x) in enumerate(p[ix, :].cpu().data.numpy())]

In [1]:
# scales = [(512 - 32*i, 64, 5) for i in range(5, 12)]
# for t in scales:
#     print(t)

# res_scales = defaultdict(list)
# for crop_size, step, batch_size in scales:
#     ds_test = TifFolderExFiles(
#         test_path,
#         transform=transforms.Compose([
#             transforms.Lambda(lambda img: NCrops(np.array(img), crop_size=crop_size, step=step)),
#             transforms.Lambda(lambda crops: torch.stack([normalize(to_tensor(crop)) for crop in crops]))
#         ]))

#     test_loader = torch.utils.data.DataLoader(    
#         ds_test,
#         batch_size=batch_size, 
#         shuffle=False,
#         num_workers=num_workers, 
#         pin_memory=True)
    
#     for X, Y, files in tqdm_notebook(test_loader, total=int(len(ds_test.imgs)/batch_size), desc=str(crop_size)):
#         files = list(map(lambda s: os.path.basename(s), files))
#         bs, ncrops, c, h, w = X.shape
#         X = X.view(-1, c, h, w)
#         X_var = Variable(X.cuda(), volatile=True)
#         log_p = model(X_var)
#         log_p = log_p.view(bs, ncrops, -1)
#         p = F.softmax(log_p, dim=2)

#         for ix, fname in enumerate(files):
#             res_scales[fname].append(p[ix, :].cpu().data.numpy())
            
            
# res = []
# for fname, p in res_scales.items():
#     p = np.vstack(p)
#     if (p == 0).sum() > 0:
#         p[p == 0] += 1e-16
#     c_id = np.exp(np.log(p).mean(axis=0)).argmax()
#     res.append((fname, idx2class[c_id]))

In [7]:
# crop_size = 256
# step = 64
# batch_size = 5

# ds_test = TifFolderExFiles(
#     test_path,
#     transform=transforms.Compose([
#         transforms.Lambda(lambda img: NCrops(np.array(img), crop_size=crop_size, step=step)),
#         transforms.Lambda(lambda crops: torch.stack([normalize(to_tensor(crop)) for crop in crops]))
#     ]))

# test_loader = torch.utils.data.DataLoader(    
#     ds_test,
#     batch_size=batch_size, 
#     shuffle=False,
#     num_workers=num_workers, 
#     pin_memory=True)

# res = []
# p_test = {}
# for X, Y, files in tqdm_notebook(test_loader, total=int(len(ds_test.imgs)/batch_size)):
#     files = list(map(lambda s: os.path.basename(s), files))
#     bs, ncrops, c, h, w = X.shape
#     X = X.view(-1, c, h, w)
#     X_var = Variable(X.cuda(), volatile=True)
#     log_p = model(X_var)
#     log_p = log_p.view(bs, ncrops, -1)
#     p = F.softmax(log_p, dim=2) #.mean(dim=1)
#     p = p.prod(dim=1).pow(1/p.shape[1])
#     ix_argmax = p.max(1)[1].cpu().data.numpy()
#     res.extend(list(zip(files, [idx2class[ix] for ix in ix_argmax])))

#     for ix in range(len(files)):
#         p_test[files[ix]] = [(idx2class[i], x) for (i, x) in enumerate(p[ix, :].cpu().data.numpy())]

In [46]:
with open(os.path.join(out_dir, 'submit__unalt.csv'.lower()), 'w') as f:
    f.write('fname,camera\n')
    for fname, c in res:
        if '_unalt' in fname:
            f.write('%s,%s\n' % (fname, c))
        else:
            f.write('%s,%s\n' % (fname, 'no_class'))
            
            
with open(os.path.join(out_dir, 'submit__manip.csv'.lower()), 'w') as f:
    f.write('fname,camera\n')
    for fname, c in res:
        if '_manip' in fname:
            f.write('%s,%s\n' % (fname, c))
        else:
            f.write('%s,%s\n' % (fname, 'no_class'))
            
            
with open(os.path.join(out_dir, 'submit.csv'.lower()), 'w') as f:
    f.write('fname,camera\n')
    for fname, c in res:
        f.write('%s,%s\n' % (fname, c))